# Azure Language - Sentiment Analysis

This notebook demonstrates how to use the Azure Language Analyze Text API to perform
sentiment analysis on customer reviews for a restaurant called "The Bowl".

## Prerequisites

Set the following environment variables:
- `FOUNDRY_ENDPOINT` - Azure Language service endpoint
- `FOUNDRY_API_KEY` - API key for authentication

In [ ]:
import json
import os

import requests
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

## Load Sample Data

Load customer reviews from the sample data file.

In [ ]:
with open("01-SentimentAnalysis-data/restaurant_reviews.txt", encoding="utf-8") as f:
    raw_text = f.read()

# Parse individual reviews (separated by blank lines)
reviews = []
current_review = []
for line in raw_text.split("\n"):
    if line.strip() == "":
        if current_review:
            reviews.append("\n".join(current_review))
            current_review = []
    else:
        current_review.append(line)
if current_review:
    reviews.append("\n".join(current_review))

print(f"Loaded {len(reviews)} reviews")
for i, review in enumerate(reviews[:3], 1):
    print(f"\nReview {i} preview: {review[:100]}...")

## API Setup

Configure the Azure Language API endpoint and authentication.

In [ ]:
endpoint = os.environ["FOUNDRY_ENDPOINT"]
api_key = os.environ["FOUNDRY_API_KEY"]

# Construct the Analyze Text API URL
api_url = f"{endpoint}/language/:analyze-text?api-version=2024-11-01"

headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
}

print(f"API Endpoint: {endpoint}")
print("API Key: [CONFIGURED]")

## Make API Call

Send the reviews to the Sentiment Analysis API.

In [ ]:
# Prepare documents for the API
documents = [
    {"id": str(i + 1), "language": "en", "text": review}
    for i, review in enumerate(reviews)
]

# Build the request payload
payload = {
    "kind": "SentimentAnalysis",
    "parameters": {"modelVersion": "latest"},
    "analysisInput": {"documents": documents},
}

# Make the API request
response = requests.post(api_url, headers=headers, json=payload, timeout=30)
response.raise_for_status()
result = response.json()

print("API call successful!")

## Raw API Results

Display the complete JSON response from the API.

In [ ]:
print(json.dumps(result, indent=2))

## Human-Friendly Results

Present the sentiment analysis results in a readable format.

In [ ]:
console = Console()

# Create a summary table
table = Table(title="Sentiment Analysis Results - The Bowl Restaurant Reviews")
table.add_column("Review #", style="cyan", justify="center")
table.add_column("Sentiment", style="bold", justify="center")
table.add_column("Confidence", justify="center")
table.add_column("Preview", style="dim", max_width=50)


def get_sentiment_style(sentiment: str) -> str:
    """Return color style based on sentiment."""
    styles = {
        "positive": "green",
        "negative": "red",
        "neutral": "yellow",
        "mixed": "magenta",
    }
    return styles.get(sentiment.lower(), "white")


# Process each document result
for doc in result["results"]["documents"]:
    doc_id = doc["id"]
    sentiment = doc["sentiment"]
    confidence = doc["confidenceScores"]

    # Find the original review text
    review_idx = int(doc_id) - 1
    preview = reviews[review_idx][:50].replace("\n", " ") + "..."

    # Format confidence scores
    conf_str = f"P:{confidence['positive']:.0%} N:{confidence['negative']:.0%}"

    style = get_sentiment_style(sentiment)
    table.add_row(doc_id, f"[{style}]{sentiment.upper()}[/{style}]", conf_str, preview)

console.print(table)

# Summary statistics
sentiments = [doc["sentiment"] for doc in result["results"]["documents"]]
positive_count = sum(1 for s in sentiments if s == "positive")
negative_count = sum(1 for s in sentiments if s == "negative")
neutral_count = sum(1 for s in sentiments if s == "neutral")
mixed_count = sum(1 for s in sentiments if s == "mixed")

summary = f"""
[bold]Summary for The Bowl Restaurant:[/bold]
• Total Reviews: {len(sentiments)}
• [green]Positive:[/green] {positive_count}
• [red]Negative:[/red] {negative_count}
• [yellow]Neutral:[/yellow] {neutral_count}
• [magenta]Mixed:[/magenta] {mixed_count}
"""

console.print(Panel(summary, title="Review Analysis Summary"))